---

# Atenção!

Lembre-se de clonar este notebook antes de tentar editar as células de código. Basta seguir os passos:

File -> Save a copy in Drive



---

In [ ]:
!git clone https://github.com/loggi/loggibud
%cd /content/loggibud/

# Instale as dependências do projeto
!pip install poetry
!poetry install
# Se você estiver executando esse script localmente, não precisa dos dois comandos abaixo
!poetry export -f requirements.txt --without-hashes --output requirements.txt
!pip install -r requirements.txt

# Verifique se tudo funcionou executando os testes
!poetry run pytest -s -v tests/

# Baixe os dados compilados
!wget -nc https://loggibud.s3.amazonaws.com/dataset.zip
!unzip -n dataset.zip

# Verifique que a pasta `data/` agora não está mais vazia
!ls data/

Cloning into 'loggibud'...
remote: Enumerating objects: 938, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 938 (delta 36), reused 45 (delta 27), pack-reused 871
Receiving objects: 100% (938/938), 6.40 MiB | 28.88 MiB/s, done.
Resolving deltas: 100% (493/493), done.
/content/loggibud
     |████████████████████████████████| 174kB 6.7MB/s 
     |████████████████████████████████| 7.2MB 8.9MB/s 
     |████████████████████████████████| 430kB 45.9MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 92kB 9.3MB/s 
     |████████████████████████████████| 348kB 32.4MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 3.2MB 35.7MB/s 
  Found existing installation: importlib-metadata 4.5.0
    Uninstalling importlib-metadata-4.5.0:
      Successfully uninstalled importlib-metadata-4.5.0
Creating virtualenv loggibud-3SbdMl6d-py3.7 in /root

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1 -- /root/.cache/pypoetry/virtualenvs/loggibud-3SbdMl6d-py3.7/bin/python
cachedir: .pytest_cache
rootdir: /content/loggibud
collected 7 items / 1 skipped / 6 selected                                     

tests/v1/test_data_conversion.py::test_can_create_proper_tsplib_from_instance PASSED
tests/v1/test_distances.py::test_great_circle_distance PASSED
tests/v1/test_distances.py::test_great_circle_route_distance PASSED
tests/v1/test_task1_baselines.py::test_ortools_solver PASSED
tests/v1/test_task1_baselines.py::test_lkh_solver PASSED
100% 158/158 [00:00<00:00, 2500.02it/s]
PASSED
100% 158/158 [00:00<00:00, 67525.99it/s]
PASSED

========================= 7 passed, 1 skipped in 8.85s =========================
--2021-06-22 12:49:49--  https://loggibud.s3.amazonaws.com/dataset.zip
Resolving loggibud.s3.amazonaws.com (loggibud.s3.amazonaws.com

In [ ]:
# Arquivo de configuração para acessar o OSRM
from loggibud.v1.distances import OSRMConfig


osrm_config = OSRMConfig(host="http://ec2-34-222-175-250.us-west-2.compute.amazonaws.com")
osrm_config

OSRMConfig(host='http://ec2-34-222-175-250.us-west-2.compute.amazonaws.com', timeout_s=600)

Aqui está o nosso código completo em K-means:

In [ ]:
# ============================================================================ #
# Solver K-means, etapa de execução
# ============================================================================ #
import numpy as np
from ortools.constraint_solver import pywrapcp
from sklearn.cluster import KMeans

from loggibud.v1.distances import calculate_distance_matrix_m, OSRMConfig
from loggibud.v1.types import CVRPInstance, CVRPSolution, CVRPSolutionVehicle


def solve_vrp_kmeans(problem, model):
    """"""
    # Inicializa o dicionário com sub-rotas
    num_clusters = model.n_clusters
    subroutes = {}
    for i in range(num_clusters):
        subroutes[i] = []

    # Inicializa a variável com os veículos completos
    vehicles = []

    # Resolve o problema dinamicamente
    for delivery in problem.deliveries:
        route(problem, model, delivery, subroutes, vehicles)

    # Para cada sub-rota sobrando em `subroutes`, construa um novo veículo
    for subroute in subroutes.values():
        if subroute:
            vehicle_solution = _construct_vehicle(problem, subroute)
            vehicles.append(vehicle_solution)

    # Ao final, retorne uma variável do tipo `CVRPSolution`
    return CVRPSolution(name=problem.name, vehicles=vehicles)


def route(problem, model, delivery, subroutes, vehicles):
    """"""
    delivery_point = np.array([(delivery.point.lat, delivery.point.lng)])
    subregion_index = model.predict(delivery_point)[0]

    # Verifica se o veículo em `subregion_index` comporta o novo pacote
    if (
        _compute_vehicle_volume(subroutes[subregion_index]) + delivery.size
        <= problem.vehicle_capacity
    ):
        subroutes[subregion_index].append(delivery)
    else:
        # Finaliza a rota atual
        vehicle_solution = _construct_vehicle(
            problem, subroutes[subregion_index]
        )
        vehicles.append(vehicle_solution)

        # Adiciona um novo veículo a esta sub-região
        subroutes[subregion_index] = [delivery]


def _compute_vehicle_volume(deliveries):
    volume = 0
    for delivery in deliveries:
        volume += delivery.size
    return volume


def _construct_vehicle(problem, vehicle_deliveries):
    distance_matrix = _compute_distance_matrix(problem, vehicle_deliveries)
    ordered_indices, _ = solve_tsp_ortools(distance_matrix)

    # `ordered_indices` tem o formato `[0, 4, 3, 1, ..., 0]`. Precisamos
    # remover a origem do início e do final e ordenar a variável
    # `vehicle_deliveries` usando estes índices
    ordered_vehicle_deliveries = []
    for ordered_index in ordered_indices[1:-1]:
        ordered_vehicle_deliveries.append(
            vehicle_deliveries[ordered_index - 1]
        )

    return CVRPSolutionVehicle(
        origin=problem.origin, deliveries=ordered_vehicle_deliveries
    )


def _compute_distance_matrix(problem, vehicle_deliveries):
    # Os pontos do problema consistem na origem mais as entregas em
    # `vehicle_deliveries`
    points = [problem.origin]
    for delivery in vehicle_deliveries:
        points.append(delivery.point)

    # Configuração com o servidor para os alunos
    return calculate_distance_matrix_m(points, config=osrm_config)


def solve_tsp_ortools(distance_matrix):
    n = distance_matrix.shape[0]  # número de nós do problema
    num_vehicles = 1  # número de veículos (no nosso caso, apenas um)
    depot_node = 0  # número do nó que representa o ponto de origem
    manager = pywrapcp.RoutingIndexManager(n, num_vehicles, depot_node)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(i, j):
        # `i` e `j` são índices internos do OR-Tools. Precisamos primeiro
        # convertê-los em nós do nosso problema
        ni = manager.IndexToNode(i)
        nj = manager.IndexToNode(j)
        return distance_matrix[ni, nj]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Resolve o problema com métodos default
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    solution = routing.SolveWithParameters(search_parameters)

    # Constroi a rota final
    route = []
    index = routing.Start(0)
    node = manager.IndexToNode(index)
    route.append(node)

    while not routing.IsEnd(index):
        index = solution.Value(routing.NextVar(index))
        node = manager.IndexToNode(index)
        route.append(node)

    return route, solution.ObjectiveValue()

# Exercício 1

Aqui está uma funçao que recebe um caminho com instâncias de treinamento e o número desejado de sub-regiões para dividir o espaço:

In [ ]:
from pathlib import Path

from sklearn.cluster import KMeans
import numpy as np

from loggibud.v1.types import CVRPInstance

def vrp_kmeans_pretrain(path_str, n_clusters):
    """Treina um modelo K-means a partir de instâncias em uma dada pasta"""

    instances = _load_instances(path_str)
    model = pretrain(instances, n_clusters)
    return model


def _load_instances(train_path_str):
    # Converte o nome da pasta em uma variável `Path`
    train_path = Path(train_path_str)

    instances = []
    for instance_file in train_path.iterdir():
        instance = CVRPInstance.from_file(instance_file)
        instances.append(instance)

    return instances


def pretrain(instances, n_clusters):
    # Busque os pontos de cada entrega e combine todos em uma lista
    all_points_list = []
    for instance in instances:
        all_points_list.extend(_get_delivery_coordinates(instance))

    # Converta a lista em uma matriz para o `KMeans`
    points = np.array(all_points_list)

    # Constrói o modelo com KMeans
    model = KMeans(n_clusters=n_clusters, random_state=0).fit(points)
    return model


def _get_delivery_coordinates(instance):
    """Extrai as coordenadas de uma entrega"""
    points = []
    for delivery in instance.deliveries:
        points.append((delivery.point.lat, delivery.point.lng))

    # Converte a lista de pontos em uma matrix numpy
    return points

Vamos testar com a pasta `"./data/cvrp-instances-1.0/train/df-0/cvrp-0-df-0.json"` usando K = 10:

In [ ]:
path_str = "./data/cvrp-instances-1.0/train/df-0"
model = vrp_kmeans_pretrain(path_str, 10)
model

KMeans(n_clusters=10, random_state=0)

In [ ]:
# Vamos verificar os centroides
model.cluster_centers_

array([[-15.67286511, -47.84016927],
       [-15.61489605, -47.65774402],
       [-15.65010054, -47.79041778],
       [-15.75682328, -47.7723672 ],
       [-15.94982208, -47.47194163],
       [-15.60197792, -47.95069705],
       [-15.81947842, -47.59618473],
       [-15.63399696, -47.83629212],
       [-15.65191987, -47.64619759],
       [-15.62850033, -47.48069883]])

Em geral, nosso método parece funcionar corretamente:

In [ ]:
path_str = "./data/cvrp-instances-1.0/train/df-0"
model = vrp_kmeans_pretrain(path_str, 10)
print(f"Este modelo possui {model.cluster_centers_.shape[0]} centroides")

model = vrp_kmeans_pretrain(path_str, 50)
print(f"Este modelo possui {model.cluster_centers_.shape[0]} centroides")

model = vrp_kmeans_pretrain(path_str, 100)
print(f"Este modelo possui {model.cluster_centers_.shape[0]} centroides")

Este modelo possui 10 centroides
Este modelo possui 50 centroides
Este modelo possui 100 centroides


# Exercício 2

Vamos inicialmente analisar a instância a ser resolvida:

In [ ]:
from loggibud.v1.types import CVRPInstance
from loggibud.v1.plotting.plot_instance import plot_cvrp_instance

problem = CVRPInstance.from_file("./data/cvrp-instances-1.0/dev/pa-0/cvrp-0-pa-100.json")
plot_cvrp_instance(problem)

Usando o algoritmo de antes e a pasta `"./data/cvrp-instances-1.0/train/pa-0"`, podemos treinar um modelo com 10 sub-regiões:

In [ ]:
path_str = "./data/cvrp-instances-1.0/train/pa-0"
model = vrp_kmeans_pretrain(path_str, 10)
model

KMeans(n_clusters=10, random_state=0)

Agora usamos o solver `solve_vrp_kmeans` com a instância desejada:

In [ ]:
solution = solve_vrp_kmeans(problem, model)
solution

CVRPSolution(name='cvrp-0-pa-100', vehicles=[CVRPSolutionVehicle(origin=Point(lng=-47.93317089927402, lat=-1.2923387357484621), deliveries=[Delivery(id='59938619055ccc1e7ca6a92f811e9e', point=Point(lng=-47.929271177599, lat=-1.2859786943179035), size=1), Delivery(id='38000ae55b1f06e657c87966e2f1a6e6', point=Point(lng=-47.93007357958298, lat=-1.2822331057939618), size=4), Delivery(id='49f4ce8b674d7c56de3bba91312a7203', point=Point(lng=-47.93056863252913, lat=-1.282537296710896), size=10), Delivery(id='a426f00d4d426615bd40047c4dc34154', point=Point(lng=-47.931970559926164, lat=-1.2797694169860556), size=3), Delivery(id='99cec9edd67ff9d8f6ac11e1393c895b', point=Point(lng=-47.92752568586827, lat=-1.279002722859885), size=1), Delivery(id='6d512dee3a7df7b8e1a004e91a54aec7', point=Point(lng=-47.92327860000773, lat=-1.276954830612779), size=7), Delivery(id='dae5ed59ae56d290d6ef90095116a887', point=Point(lng=-47.92212541462644, lat=-1.277506448703492), size=6), Delivery(id='346738ec031daba39565

In [ ]:
from loggibud.v1.eval.task1 import evaluate_solution

# Verifique que a solução é factível
evaluate_solution(problem, solution, config=osrm_config)

901.9313

In [ ]:
from loggibud.v1.plotting.plot_solution import plot_cvrp_solution


plot_cvrp_solution(solution)

# Exercício 3

Vamos usar as funções dos exercícios anteriores para criar 4 modelos, cada um com um número de sub-regiões:

In [ ]:
path_str = "./data/cvrp-instances-1.0/train/df-0"

model_10 = vrp_kmeans_pretrain(path_str, 10)
print(f"Este modelo tem {model_10.cluster_centers_.shape[0]} centroides")

model_30 = vrp_kmeans_pretrain(path_str, 30)
print(f"Este modelo tem {model_30.cluster_centers_.shape[0]} centroides")

model_50 = vrp_kmeans_pretrain(path_str, 50)
print(f"Este modelo tem {model_50.cluster_centers_.shape[0]} centroides")

model_100 = vrp_kmeans_pretrain(path_str, 100)
print(f"Este modelo tem {model_100.cluster_centers_.shape[0]} centroides")

Este modelo tem 10 centroides
Este modelo tem 30 centroides
Este modelo tem 50 centroides
Este modelo tem 100 centroides


Agora, vamos resolver o problema `"cvrp-0-df-90.json"` em cada modelo:

In [ ]:
from loggibud.v1.types import CVRPInstance


problem = CVRPInstance.from_file("./data/cvrp-instances-1.0/dev/df-0/cvrp-0-df-90.json")

In [ ]:
solution_10 = solve_vrp_kmeans(problem, model_10)
solution_30 = solve_vrp_kmeans(problem, model_30)
solution_50 = solve_vrp_kmeans(problem, model_50)
solution_100 = solve_vrp_kmeans(problem, model_100)

In [ ]:
from loggibud.v1.eval.task1 import evaluate_solution


print(f"Model with K = 10 has solution {evaluate_solution(problem, solution_10, config=osrm_config)}")
print(f"Model with K = 30 has solution {evaluate_solution(problem, solution_30, config=osrm_config)}")
print(f"Model with K = 50 has solution {evaluate_solution(problem, solution_50, config=osrm_config)}")
print(f"Model with K = 100 has solution {evaluate_solution(problem, solution_100, config=osrm_config)}")

Model with K = 10 has solution 2100.0867
Model with K = 30 has solution 2389.3921
Model with K = 50 has solution 2934.4384
Model with K = 100 has solution 3820.5852


Vemos aqui que o quanto menor o valor de K menor é a distância total das rotas.

Mas será que isso é uma regra?

Vamos experimentar com K = 3:

In [ ]:
model_3 = vrp_kmeans_pretrain(path_str, 3)
solution_3 = solve_vrp_kmeans(problem, model_3)
print(f"Model with K = 3 has solution {evaluate_solution(problem, solution_5, config=osrm_config)}")

Model with K = 3 has solution 2132.5045


Bem, conseguimos uma solução pior do que o caso com K = 10. Então para este problema específico parece existir um número ótimo de sub-regiões entre 3 e 10.

Como determiná-lo? Fora tentativa e erro, não sabemos. Por isso este é um outro problema por si só.

# Exercício 4

Aqui está um solver que combina as funções desenvolvidas anteriormente em um solver com a estrutura sugerida. Observe como algumas variáveis que controlam as rotas atuais (como `vehicles` e `subroutes`) transformaram-se em propriedades.

In [ ]:
from dataclasses import dataclass
from typing import Dict, List, Optional

import numpy as np
from ortools.constraint_solver import pywrapcp
from sklearn.cluster import KMeans

from loggibud.v1.distances import calculate_distance_matrix_m, OSRMConfig
from loggibud.v1.types import (
    CVRPInstance, CVRPSolution, CVRPSolutionVehicle, Point
)


@dataclass
class KMeansSolver:
    model: Optional[KMeans] = None
    subroutes: Optional[Dict[int, List[int]]] = None
    vehicles: Optional[List[int]] = None
    vehicle_capacity: Optional[int] = None
    origin: Optional[Point] = None
    n_clusters: int = 10
    osrm_config: OSRMConfig = OSRMConfig(
        host="http://ec2-34-222-175-250.us-west-2.compute.amazonaws.com"
    )

    def pretrain(self, train_instances):
        # Treina um KMeans com os dados de entrada
        # Busque os pontos de cada entrega e combine todos em uma lista
        all_points_list = []
        for instance in train_instances:
            all_points_list.extend(_get_delivery_coordinates(instance))

        # Converta a lista em uma matriz para o `KMeans`
        points = np.array(all_points_list)

        # Constrói o modelo com KMeans
        self.model = KMeans(
            n_clusters=self.n_clusters, random_state=0
        ).fit(points)

        # Inicializa informações do problema atual
        # Inicializa o dicionário com sub-rotas
        subroutes = {}
        for i in range(self.n_clusters):
            subroutes[i] = []

        # Inicializa a variável com os veículos completos
        vehicles = []

        self.subroutes = subroutes
        self.vehicles = vehicles

        # Armazena propriedades do problema
        self.vehicle_capacity = train_instances[0].vehicle_capacity
        self.origin = train_instances[0].origin
        self.name = train_instances[0].name

    def finetune(self, delivery):
        """Nada é feito aqui, logo, deixe o código em branco"""
        pass

    def route(self, delivery):
        delivery_point = np.array([(delivery.point.lat, delivery.point.lng)])
        subregion_index = self.model.predict(delivery_point)[0]

        # Verifica se o veículo em `subregion_index` comporta o novo pacote
        if (
            _compute_vehicle_volume(self.subroutes[subregion_index])
            + delivery.size
            <= self.vehicle_capacity
        ):
            self.subroutes[subregion_index].append(delivery)
        else:
            # Finaliza a rota atual
            vehicle_solution = self._construct_vehicle(
                self.subroutes[subregion_index]
            )
            self.vehicles.append(vehicle_solution)

            # Adiciona um novo veículo a esta sub-região
            self.subroutes[subregion_index] = [delivery]

    def _construct_vehicle(self, vehicle_deliveries):
        distance_matrix = self._compute_distance_matrix(vehicle_deliveries)
        ordered_indices, _ = solve_tsp_ortools(distance_matrix)

        # `ordered_indices` tem o formato `[0, 4, 3, 1, ..., 0]`. Precisamos
        # remover a origem do início e do final e ordenar a variável
        # `vehicle_deliveries` usando estes índices
        ordered_vehicle_deliveries = []
        for ordered_index in ordered_indices[1:-1]:
            ordered_vehicle_deliveries.append(
                vehicle_deliveries[ordered_index - 1]
            )

        return CVRPSolutionVehicle(
            origin=self.origin, deliveries=ordered_vehicle_deliveries
        )

    def _compute_distance_matrix(self, vehicle_deliveries):
        # Os pontos do problema consistem na origem mais as entregas em
        # `vehicle_deliveries`
        points = [self.origin]
        for delivery in vehicle_deliveries:
            points.append(delivery.point)

        # Configuração com o servidor para os alunos
        return calculate_distance_matrix_m(points, config=self.osrm_config)

    def finish(self):
        # Para cada sub-rota sobrando em `subroutes`, construa um novo veículo
        for subroute in self.subroutes.values():
            vehicle_solution = self._construct_vehicle(subroute)
            self.vehicles.append(vehicle_solution)

        # Ao final, retorne uma variável do tipo `CVRPSolution`
        return CVRPSolution(name=self.name, vehicles=self.vehicles)


def _get_delivery_coordinates(instance):
    """Extrai as coordenadas de uma entrega"""
    points = []
    for delivery in instance.deliveries:
        points.append((delivery.point.lat, delivery.point.lng))

    # Converte a lista de pontos em uma matrix numpy
    return points


def _compute_vehicle_volume(deliveries):
    volume = 0
    for delivery in deliveries:
        volume += delivery.size
    return volume


def solve_tsp_ortools(distance_matrix):
    n = distance_matrix.shape[0]  # número de nós do problema
    num_vehicles = 1  # número de veículos (no nosso caso, apenas um)
    depot_node = 0  # número do nó que representa o ponto de origem
    manager = pywrapcp.RoutingIndexManager(n, num_vehicles, depot_node)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(i, j):
        # `i` e `j` são índices internos do OR-Tools. Precisamos primeiro
        # convertê-los em nós do nosso problema
        ni = manager.IndexToNode(i)
        nj = manager.IndexToNode(j)
        return distance_matrix[ni, nj]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Resolve o problema com métodos default
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    solution = routing.SolveWithParameters(search_parameters)

    # Constroi a rota final
    route = []
    index = routing.Start(0)
    node = manager.IndexToNode(index)
    route.append(node)

    while not routing.IsEnd(index):
        index = solution.Value(routing.NextVar(index))
        node = manager.IndexToNode(index)
        route.append(node)

    return route, solution.ObjectiveValue()

In [ ]:
# Aqui está a função que resolve problemas com um solver na estrutura acima
def solve(train_instances, problem):
    solver = KMeansSolver()

    # Treina o solver com as instancias conhecidas
    solver.pretrain(train_instances)

    for delivery in problem.deliveries:
        solver.finetune(delivery)
        solver.route(delivery)

    # Retorna uma solution na forma `CVRPSolution`
    return solver.finish()

Para exemplificar, vamos testar com a instância

In [ ]:
from loggibud.v1.types import CVRPInstance


def _load_instances(train_path_str):
    # Converte o nome da pasta em uma variável `Path`
    train_path = Path(train_path_str)

    instances = []
    for instance_file in train_path.iterdir():
        instance = CVRPInstance.from_file(instance_file)
        instances.append(instance)

    return instances

# Carrega as instâncias de treino
train_path_str = "./data/cvrp-instances-1.0/train/pa-0"
train_instances = _load_instances(train_path_str)

# Carrega a instância problema
problem = CVRPInstance.from_file("./data/cvrp-instances-1.0/dev/pa-0/cvrp-0-pa-100.json")

# Resolve o problema
solution = solve(train_instances, problem)
solution

CVRPSolution(name='cvrp-0-pa-77', vehicles=[CVRPSolutionVehicle(origin=Point(lng=-47.93317089927402, lat=-1.2923387357484621), deliveries=[Delivery(id='59938619055ccc1e7ca6a92f811e9e', point=Point(lng=-47.929271177599, lat=-1.2859786943179035), size=1), Delivery(id='38000ae55b1f06e657c87966e2f1a6e6', point=Point(lng=-47.93007357958298, lat=-1.2822331057939618), size=4), Delivery(id='49f4ce8b674d7c56de3bba91312a7203', point=Point(lng=-47.93056863252913, lat=-1.282537296710896), size=10), Delivery(id='a426f00d4d426615bd40047c4dc34154', point=Point(lng=-47.931970559926164, lat=-1.2797694169860556), size=3), Delivery(id='99cec9edd67ff9d8f6ac11e1393c895b', point=Point(lng=-47.92752568586827, lat=-1.279002722859885), size=1), Delivery(id='6d512dee3a7df7b8e1a004e91a54aec7', point=Point(lng=-47.92327860000773, lat=-1.276954830612779), size=7), Delivery(id='dae5ed59ae56d290d6ef90095116a887', point=Point(lng=-47.92212541462644, lat=-1.277506448703492), size=6), Delivery(id='346738ec031daba395657

In [ ]:
from loggibud.v1.eval.task1 import evaluate_solution

# Verifique que a solução é factível
evaluate_solution(problem, solution, config=osrm_config)

901.9313

Veja que o resultado é o mesmo, afinal, é o mesmo algoritmo, apenas escrito de outra forma.

In [ ]:
from loggibud.v1.plotting.plot_solution import plot_cvrp_solution


plot_cvrp_solution(solution)